In [1]:
flex_home = "https://josefmtd.com"
flex_title = "South Sumatra Fire Danger Rating System"
flex_subtitle = "Josef Matondang - 2021"

In [4]:
import ee
import folium
import geemap.foliumap as geemap
from datetime import date, timedelta

# Interactive Python Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

min_dict = {
    'FWI' : 1.0, 'ISI' : 1.0, 'BUI' : 6.0,
    'DC' : 140.0, 'DMC' : 4.0, 'FFMC' : 72.0,
}

max_dict = {
    'FWI' : 13.0, 'ISI' : 5.0, 'BUI' : 33.0,
    'DC' : 350.0, 'DMC' : 29.0, 'FFMC' : 82.0,
}

## Section

### Parameters

In [3]:
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=1)
)

comp_label = widgets.Label(value='Component:')
comp_var = widgets.Dropdown(
    options=[
        'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI'
    ],
    value='FWI',
)

widgets.VBox([obs_label, obs_var, comp_label, comp_var])

## Column

### Fire Weather Index

In [4]:
out = widgets.Output()

In [5]:
def on_value_change(change):
    obs = obs_var.value
    comp = comp_var.value

    with out:
        fig = folium.Figure(height="100%")
        Map = geemap.Map(center=(-3.1, 104.1), zoom=8,
            add_google_map=False).add_to(fig)

        url = f'gs://fwi-bicubic-outputs/ID-SS_{obs.isoformat()}_FWI.tif'
        fwi_data = ee.Image.loadGeoTIFF(url)
        palette = ['blue', 'green', 'yellow', 'red']

        min_val = min_dict[comp]
        max_val = max_dict[comp]

        vis_params = {
            'min': min_val, 
            'max': max_val, 
            'palette': palette
        }

        sumatera_selatan = ee.FeatureCollection('FAO/GAUL/2015/level2') \
            .filter(ee.Filter.eq('ADM1_NAME', 'Sumatera Selatan'))

        Map.addLayer(fwi_data.select(comp), vis_params, comp)
        Map.addLayer(sumatera_selatan, {'fillColor' : 'ffffff'}, 'South Sumatra')
        Map.add_colorbar(colors=palette, vmin=min_val, vmax=max_val)
        clear_output(wait=True)
        display(Map)

obs_var.observe(on_value_change, names="value")
comp_var.observe(on_value_change, names="value")

In [6]:
on_value_change(None)
out

Output()